**Noman Aziz**
---
**i181561@nu.edu.pk**


#Documentation

1.   Data Loading
  *   I load data.txt and misspellings.txt into python lists using google drive.
2.   Uni-Gram Model Training
  *   Then i generate my language model i.e, i generate unigrams using data list and calculate their probabilities and store them in python dictionary data structure.
3.   Error Tables Generation
  *   Then i generate empty python dictionary which contain tuples as key.
  *   I start from ('#','#'), ('#','a') to ('z','y'), ('z','z') and assign these keys 0 value
  *   Then i populate these dictionaries using misspellings.txt file. I determine the type of edit between wrong and correct word, and update the specific edit table value accordingly.
4.    P(x|w) calculation
  *    First i generate a Chars list which contain count of individual english letters in the alphabet along with '#'. It is stored in python dictionary
  *    Then i generate a CharsDouble list which contain count of bigram (two) english letters in the alphabet along with '#'. It is stored in python dictionary as a tuple.
  *    Then i use the formula provided in noisy-channel.pdf file to calculate P(x|w). I pass incorrect letter, correct letter and type of edit to the function.
5.    Calculation of List of Candidate Words
  *    I generate list of candidate words from the typed word using edit distance formula
  *    I used Damerau–Levenshtein distance for the edit distance
  *    I improved the performance of this function by maintaining a memory which keeps count of the words whose edit distance was already calculated so it avoids repeating recursive iterations
  *    My fain candidate generation function use this edit distance function and add only those words in the list who has edit distance of 1 or 0
6.    Main Function
  *    It first sends a typed word to candidate word function which generate a list of candidate words
  *    if there are no candidate words, then the typed word will remain as it is (no auto correction)
  *    If there are candidate words, it sends the list and the typed word to the suggested word function.
  *    the suggested word function iterates over the candidate word list, calculate the editType and letter1 and letter2 of that particular edit. Then it calculates P(x|w) using previous defined function and multiply it with P(w) which we get from the unigrams dictionary.
  *    This function returns the candidate word with highest probability and if a case occurs, in which the edit distance was 0, then it assumes that the typed word is the correct word and returns it back since it has the highest probability.

#Importing Libraries

In [ ]:
from google.colab import files, drive
import csv
import itertools

#Importing Data

In [ ]:
#Loading File from Uploading
#uploadedFiles = files.upload()

#Loading Files from Drive
drive.mount('/content/drive')

#Loading Data.txt file
with open('/content/drive/My Drive/data.txt', 'rt', encoding="utf-8") as f:
  reader = csv.reader(f)
  data = list(reader)
f.close()

#Loading Misspellings.txt file
with open('/content/drive/My Drive/misspellings.txt', 'rt', encoding="utf-8") as f:
  reader = csv.reader(f)
  misSpellings = list(reader)
f.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Language Model

In [ ]:
unigrams = dict()

def genUnigrams():
  lenCorpus = 0
  for line in data:
    splittedWords = line[0].split(' ')
    for word in splittedWords:
      lenCorpus += 1
      if word not in unigrams:
        unigrams[word] = 1
      else:
        unigrams[word] += 1

  #Assigning Probabilities to Unigrams
  for key in unigrams:
    unigrams[key] = (unigrams[key] / lenCorpus)

genUnigrams()

#Error Model

##Generating Initial Insert, Delete, Subsitute and Transpose Tables

In [ ]:
#Generating Initial Insert, Delete, Subsitute and Transpose Tables
def genTable():
  dictionary = dict()
  alphabets = ['#', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
  for letter1 in alphabets:
    for letter2 in alphabets:
      combination = (letter1, letter2)
      dictionary[combination] = 0
  return dictionary

insertTable = genTable()
deleteTable = genTable()
subsitutionTable = genTable()
transposeTable = genTable()

##Auxilary Handler Functions for Tables

In [ ]:
def handleInsertTable(correctWord, wrongWord):
  for index, (letter1, letter2) in enumerate(itertools.zip_longest(correctWord, wrongWord)):
    #Insertion at Beginning
    if (index == 0) and (letter1 != letter2):
      insertTable[('#', letter2)] += 1
      return "Insertion", "#", letter2

    #Insertion at End or Middle
    elif (letter1 == None) or ((letter1 != letter2) and (index != 0)):
      insertTable[(correctWord[index-1], letter2)] += 1
      return "Insertion", correctWord[index-1], letter2

def handleDeleteTable(correctWord, wrongWord):
  for index, (letter1, letter2) in enumerate(itertools.zip_longest(correctWord, wrongWord)):
    #delete at start
    if (index==0) and (letter1 != letter2):
      deleteTable[('#', letter1)] += 1
      return "Deletion", "#", letter1

    #delete at beginning or end
    elif (index!=0) and (letter1 != letter2):
      deleteTable[(correctWord[index-1], letter1)] += 1
      return "Deletion", correctWord[index-1], letter1

def handleSubsitutionAndTransposeTable(correctWord, wrongWord):
  for index, (letter1, letter2) in enumerate(zip(correctWord, wrongWord)):
    if letter1 != letter2:
      #Transpose Case
      if (index + 1 < len(wrongWord)) and (correctWord[index] == wrongWord[index + 1]):
        transposeTable[(letter1, letter2)] += 1
        return "Transpose", letter1, letter2
      #Subsitution Case
      else:
        subsitutionTable[(letter1, letter2)] += 1
        return "Subsitution", letter1, letter2

##Determines Type of Edit

In [ ]:
def determineTypeOfEdit(wrongWord, correctWord):
    if(len(wrongWord) > len(correctWord)):
      return handleInsertTable(correctWord, wrongWord)
    elif(len(wrongWord) < len(correctWord)):
      return handleDeleteTable(correctWord, wrongWord)
    else:
      return handleSubsitutionAndTransposeTable(correctWord, wrongWord)

##Populating Tables Values from Misspellings File

In [ ]:
def populateTables():
  initialLine = True
  for line in misSpellings:
    #Ignoring the First Line
    if(initialLine):
      initialLine = False
      continue

    correctWord = line[0].strip().lower()
    wrongWords = line[1].split('\t')
    for word in wrongWords:
      wrongWord = word.strip().lower()

      #Determining Type of Edit and Updating Table Accordingly
      determineTypeOfEdit(wrongWord, correctWord)

populateTables()

#P(x|w) Calculation

##Chars (Single Letter)

In [ ]:
charsSingle = dict()

def genCharsSingle():
  charsSingle['#'] = 0
  for line in data:
    splittedWords = line[0].split(' ')
    for word in splittedWords:
      charsSingle['#'] += 1
      for letter in word.strip().lower():
        if letter not in charsSingle:
          charsSingle[letter] = 1
        else:
          charsSingle[letter] += 1

genCharsSingle()

##Chars (Double Letter)

In [ ]:
charsDouble = dict()

def genCharsDouble():
  for line in data:
    splittedWords = line[0].split(' ')
    for word in splittedWords:
      filteredWord = word.strip().lower()

      if ('#', word[0]) in charsDouble:
        charsDouble[('#', filteredWord[0])] += 1
      else:
        charsDouble[('#', filteredWord[0])] = 1

      for index in range(len(filteredWord)-1):
        if (filteredWord[index], filteredWord[index+1]) not in charsDouble:
          charsDouble[(filteredWord[index], filteredWord[index+1])] = 1
        else:
          charsDouble[(filteredWord[index], filteredWord[index+1])] += 1

genCharsDouble()

##Function to calculate P(x|w)

In [ ]:
def calcPxw(t, c, method):
  if method == "Insertion":
    return (insertTable[(c, t)] / charsSingle[c])
  elif method == "Deletion":
    return (deleteTable[(c, t)] / charsDouble[(c, t)])
  elif method == "Subsitution":
    return (subsitutionTable[(c, t)] / charsSingle[c])
  elif method == "Transpose":
    return (transposeTable[(c, t)] / charsDouble[(c, t)])

#Candidate Words

##Edit Distance Function

In [ ]:
#Implemented using Damerau–Levenshtein distance
#We Add memory to it in order to increase efficiency (performance) of our function
memory = {}
def getEditDistance(word1, word2):

  i = len(word1)
  j = len(word2)

  distances = []

  if (i == 0):
    return j
  if (j == 0):
    return i

  if (i > 0):
    #Deletion
    parameter = (word1[:-1], word2)

    if parameter not in memory:
      memory[parameter] = getEditDistance(word1[:-1], word2)

    distances.append(memory[parameter] + 1)

  if (j > 0):
    #Insertion
    parameter = (word1, word2[:-1])

    if parameter not in memory:
      memory[parameter] = getEditDistance(word1, word2[:-1])

    distances.append(memory[parameter] + 1)

  if (i > 0) and (j > 0):
    #Subsitution
    parameter = (word1[:-1], word2[:-1])

    if(word1[-1] == word2[-1]):
      cost = 0
    else:
      cost = 1

    if parameter not in memory:
      memory[parameter] = getEditDistance(word1[:-1], word2[:-1])
  
    distances.append(memory[parameter] + cost)

  if (i > 1) and (j > 1) and (word1[-1] == word2[-2]) and (word1[-2] == word2[-1]):
    #Tranposition
    parameter = (word1[:-2], word2[:-2])

    if parameter not in memory:
      memory[parameter] = getEditDistance(word1[:-2], word2[:-2])

    distances.append(memory[parameter] + 1)

  return min(distances)

##Generating Candidate Words

In [ ]:
def getCandidateWords(word):
  candidateWords = []

  for key in unigrams:
    distance = getEditDistance(word.lower(), key.lower())
    if (distance == 1) or (distance == 0):
      candidateWords.append((key, distance))

  return candidateWords

#Suggested Word

In [ ]:
def getSuggestedWord(candidateWords, typedWord):
  w = dict()
  suggestedWord = ""  
  suggestedWordProb = 0

  for word in candidateWords:
    #Same Word with Edit Distance 0, No Need to Autocorrect
    if (word[1] == 0):
      return word[0], {}

    editType, letter1, letter2 = determineTypeOfEdit(typedWord, word[0])
    Pxw = calcPxw(letter2, letter1, editType)
    Pw = unigrams[word[0]]
    print(f"Candidate Word : {word[0]}")
    print(f"\tEdit Type : {editType}\n\tLetter 1 : {letter1}\n\tLetter 2 : {letter2}")
    print(f"\tP(x|w) = {Pxw}")
    print(f"\tP(w) = {Pw}")
    print(f"\tP(x|w) * P(w) : {Pxw * Pw}")
    w[word[0]] = Pxw * Pw
    if (Pxw * Pw) > suggestedWordProb:
      suggestedWord = word[0]
      suggestedWordProb = Pxw * Pw

  return suggestedWord, w

#Main Function

In [ ]:
typedWord = "iksa"
candidateWords = getCandidateWords(typedWord)

In [ ]:
if len(candidateWords) == 0:
  suggestedWord = typedWord
  allWordsProb = dict()
else: 
  suggestedWord, allWordsProb = getSuggestedWord(candidateWords, typedWord)

Candidate Word : kisa
	Edit Type : Transpose
	Letter 1 : k
	Letter 2 : i
	P(x|w) = 6.39544985061295e-06
	P(w) = 1.7407576372259819e-07
	P(x|w) * P(w) : 1.1132928170950259e-12
Candidate Word : iska
	Edit Type : Transpose
	Letter 1 : s
	Letter 2 : k
	P(x|w) = 0.00015742907820027078
	P(w) = 1.7439226511118473e-05
	P(x|w) * P(w) : 2.7454413541711054e-09
Candidate Word : isa
	Edit Type : Insertion
	Letter 1 : i
	Letter 2 : k
	P(x|w) = 1.2430972064675275e-05
	P(w) = 2.1474619215596883e-05
	P(x|w) * P(w) : 2.669503915686237e-10
Candidate Word : ikas
	Edit Type : Transpose
	Letter 1 : a
	Letter 2 : s
	P(x|w) = 5.477846025573716e-05
	P(w) = 3.7980166630385056e-06
	P(x|w) * P(w) : 2.0804950482688225e-10
Candidate Word : ika
	Edit Type : Insertion
	Letter 1 : k
	Letter 2 : s
	P(x|w) = 7.353873076258272e-06
	P(w) = 4.589270134504861e-07
	P(x|w) * P(w) : 3.374891008181148e-12
Candidate Word : ikla
	Edit Type : Subsitution
	Letter 1 : l
	Letter 2 : s
	P(x|w) = 1.8985813444210484e-05
	P(w) = 6.804779

In [ ]:
print(f"Typed Word : {typedWord}")
print(f"Auto-Corrected Word : {suggestedWord}")
print(f"All checked words with probabilities : {allWordsProb}")

Typed Word : iksa
Auto-Corrected Word : iska
All checked words with probabilities : {'kisa': 1.1132928170950259e-12, 'iska': 2.7454413541711054e-09, 'isa': 2.669503915686237e-10, 'ikas': 2.0804950482688225e-10, 'ika': 3.374891008181148e-12, 'ikla': 1.2919428084855966e-11, 'ikha': 1.9306619356685607e-11, 'issa': 1.2489848144807448e-10, 'insa': 2.8719922988785142e-12, 'ksa': 1.950370821186266e-12, 'ikka': 1.586272163832891e-11, 'iksha': 1.2672894996506718e-12}
